In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

base_df = pd.read_csv("/home/samir/Desktop/rudraAnalytics/sub_projects/churn/data.csv")
pd.set_option("display.max_columns", None)

In [10]:
df = base_df.copy()

In [11]:
from prepare import screen_data
# screen_data(df=df, skip=['customerID'])

In [12]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)

In [13]:

def one_hot_encode(df, encode_set=[], dont_encode=[]):
    encoded_df = df.copy()
    for column in encode_set:
        if column in dont_encode:
            continue
        # print(len(df[column].unique()))
        if df[column].dtype == 'object':
            if len(df[column].unique()) == 2:
                # Binary encoding (0 and 1)
                encoded_df[column] = pd.get_dummies(df[column], drop_first=True)
            else:
                # One-hot encoding and using 0 and 1 instead of True and False
                one_hot_encoded = pd.get_dummies(df[column], prefix=column, drop_first=False)
                one_hot_encoded.columns = [f"{column}{i+1}" for i in range(one_hot_encoded.shape[1])]
                encoded_df = pd.concat([encoded_df, one_hot_encoded], axis=1)
                encoded_df.drop(column, axis=1, inplace=True)
                # print(encoded_df)

    return encoded_df

In [14]:
features = [
    'customerID',
    'gender',
    'SeniorCitizen',
    'Partner',
    'Dependents',
    'tenure',
    'PhoneService',
    'MultipleLines',
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'Contract',
    'PaperlessBilling',
    'PaymentMethod',
    'MonthlyCharges',
    'TotalCharges',
    'Churn'
]
dont_label = ['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges']

encoded_df = one_hot_encode(df.drop('customerID', axis=1), features, dont_label)

In [15]:
X = encoded_df.drop('Churn', axis=1)
y = encoded_df['Churn']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data (optional, but often beneficial for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
accuracy = model.evaluate(X_test_scaled, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/10
141/141 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.7306 - val_loss: nan - val_accuracy: 0.7507
Epoch 2/10
141/141 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7304 - val_loss: nan - val_accuracy: 0.7507
Epoch 3/10
141/141 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7304 - val_loss: nan - val_accuracy: 0.7507
Epoch 4/10
141/141 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7304 - val_loss: nan - val_accuracy: 0.7507
Epoch 5/10
141/141 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7304 - val_loss: nan - val_accuracy: 0.7507
Epoch 6/10
141/141 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7304 - val_loss: nan - val_accuracy: 0.7507
Epoch 7/10
141/141 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.7304 - val_loss: nan - val_accuracy: 0.7507
Epoch 8/10
141/141 [============================

#ACCURACY: 73.53%